Importing the libraries and Mounting Google drive

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
df_wait = pd.read_excel("/content/gdrive/MyDrive/AI4GOOD/wait-times.xlsx", sheet_name="Wait times 2008 to 2021",header = 2)

In [ ]:
df_wait['Metric'].unique()

In [ ]:
# Selecting the year 2021 
df_wait_2021 = df_wait[df_wait['Data year'] == 2021.0]
df_wait_2021.head()

In [ ]:
df_wait_2021_provincial = df_wait_2021[::]
df_wait_2021_provincial['Metric'].unique()

In [ ]:
df_wait_2021_provincial['Region'].unique()
df_wait_2021_provincial.head()

In [ ]:
df_wait_2021_provincial['Indicator result'] = pd.to_numeric(df_wait_2021_provincial['Indicator result'], errors = 'coerce')

In [ ]:
df_wait_2021_provincial['Result'] = np.where(df_wait_2021_provincial['Unit of measurement'].eq('Hours'),df_wait_2021_provincial['Indicator result']/24,df_wait_2021_provincial['Indicator result'])
df_wait_2021_provincial['Metric'].unique()

In [ ]:
df_wait_2021_provincial["Unit of measurement"].replace({'Hours':'Days'},inplace = True)

df_wait_2021_provincial['Metric'].unique()

In [ ]:
df_wait_2021_provincial.drop('Indicator result',inplace=True,axis = 1)

In [ ]:
df_wait_2021_provincial.drop('Data year',inplace=True,axis = 1)
df_wait_2021_provincial.head()

In [ ]:
df_wait_2021_provincial['Metric_Unit'] = df_wait_2021_provincial['Metric'].str.cat(df_wait_2021_provincial['Unit of measurement'],sep = '/')
df_wait_2021_provincial.drop(['Metric','Unit of measurement'],inplace=True,axis = 1)
df_wait_2021_provincial.head()

In [ ]:
df_wait_2021_provincial['Metric_Unit'].unique()

In [ ]:
df_wait_2021_provincial.columns

In [ ]:
df_wait_2021_p = df_wait_2021_provincial[df_wait_2021_provincial['Reporting level'] == 'Provincial']
df_pivoted_p = df_wait_2021_p.pivot(index=['Reporting level','Province/territory','Indicator'], columns='Metric_Unit', values='Result')\
            .reset_index()
df_pivoted_p.columns = df_pivoted_p.columns.str.replace('Volume/Number of cases','Number of cases')
df_pivoted_p.head()

In [ ]:
df_wait_2021_regional = df_wait_2021_provincial[df_wait_2021_provincial['Reporting level'] == 'Regional']
print(df_wait_2021_regional.head())
df_pivoted_r = df_wait_2021_regional.pivot(index=['Reporting level','Province/territory','Region','Indicator'], columns='Metric_Unit', values='Result')\
            .reset_index()
df_pivoted_r.columns = df_pivoted_r.columns.str.replace('Volume/Number of cases','Number of cases')
df_pivoted_r.head()

In [ ]:
df_wait_2021_national = df_wait_2021_provincial[df_wait_2021_provincial['Reporting level'] == 'National']
print(df_wait_2021_national.head())
df_pivoted_n = df_wait_2021_national.pivot(index=['Reporting level','Province/territory','Region','Indicator'], columns='Metric_Unit', values='Result')\
            .reset_index()
df_pivoted_n.columns = df_pivoted_n.columns.str.replace('Volume/Number of cases','Number of cases')
df_pivoted_n.head()

In [ ]:
! pip install XlsxWriter

In [ ]:
#Stroing the dataframe into Excel
output_file = ("wait_time_cleaned.xlsx")
writer = pd.ExcelWriter(output_file,engine = 'xlsxwriter')
df_pivoted_n.to_excel(writer,sheet_name = 'National')
df_pivoted_r.to_excel(writer,sheet_name = 'Regional')
df_pivoted_p.to_excel(writer,sheet_name = 'Provincial')
writer.save()